In [1]:
import numpy as np

In [2]:
import pandas as pd

In [3]:
import mpathic.utils as utils

In [4]:
df = pd.io.parsers.read_csv('/home/bill/test_seqs',delim_whitespace=True)

In [5]:
df

,ct,ct_0,ct_1,seq
0,1,1,0,AAA
1,2,1,1,ACC
2,3,1,2,AGG
3,2,2,0,AAT


In [4]:
import mpathic.numerics as numerics

In [7]:
seq_mat,wtrow = numerics.dataset2mutarray(df.copy(),'MAT')
print seq_mat

  (0, 4)	1
  (0, 8)	1
  (1, 5)	1
  (1, 9)	1
  (3, 4)	1
  (3, 11)	1


In [8]:
import scipy as sp
seq_mat_array = np.matrix(numerics.dataset2seqarray(df.copy(),'MAT'))
print wtrow


[ True False False False False False  True False False False  True False]


In [9]:
emat = np.zeros((4,len(df.loc[0,'seq']),len(df.index)))
seq_dict,inv_dict = utils.choose_dict('dna')
for i,s in enumerate(df['seq'].iteritems()):
    emat[:,:,i] = utils.seq2mat(s[1],seq_dict)

In [10]:
import cvxopt

In [11]:
q = seq_mat.nonzero()

In [12]:
q

(array([0, 0, 1, 1, 3, 3], dtype=int32),
 array([ 4,  8,  5,  9,  4, 11], dtype=int32))

In [13]:
emcvx = cvxopt.spmatrix(seq_mat.data,q[0],q[1])

In [14]:
seq_mat.indices

array([ 4,  8,  5,  9,  4, 11], dtype=int32)

In [15]:
seq_mat.indptr

array([0, 2, 4, 4, 6], dtype=int32)

In [16]:
seq_mat.indices

array([ 4,  8,  5,  9,  4, 11], dtype=int32)

In [17]:
seq_mat.indptr

array([0, 2, 4, 4, 6], dtype=int32)

In [18]:
seq_mat.toarray()

array([[0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1]])

In [34]:
np.multiply(np.identity(5),np.transpose(np.matrix([1,2,3,4,5])))

matrix([[ 1.,  0.,  0.,  0.,  0.],
        [ 0.,  2.,  0.,  0.,  0.],
        [ 0.,  0.,  3.,  0.,  0.],
        [ 0.,  0.,  0.,  4.,  0.],
        [ 0.,  0.,  0.,  0.,  5.]])

In [29]:
print(2*emcvx)
print cvxopt.mul(2*emcvx,3*emcvx)

[    0         0         0         0      2.00e+00     0         0     ... ]
[    0         0         0         0         0      2.00e+00     0     ... ]
[    0         0         0         0         0         0         0     ... ]
[    0         0         0         0      2.00e+00     0         0     ... ]

[    0         0         0         0      6.00e+00     0         0     ... ]
[    0         0         0         0         0      6.00e+00     0     ... ]
[    0         0         0         0         0         0         0     ... ]
[    0         0         0         0      6.00e+00     0         0     ... ]



    emcvx.todense()

In [19]:
seq_mat.indices

array([ 4,  8,  5,  9,  4, 11], dtype=int32)

In [43]:
from cvxopt import solvers, matrix, spdiag, sqrt, div, exp
import scipy
import mpmath
def robls(s, wtrow, N0, Nsm,tm,bins=1):
    N0_matrix = np.matrix(N0)
    tm_matrix = np.matrix(tm)
    Nsm_matrix = np.matrix(Nsm)
    tm_matrix_squared = np.matrix(tm**2)
    i,c = s.shape
    print i,c
    #create s matrix, the elements of this matrix are delta_s@i * delta s2@j
    #we will need this for hessian.
    
    s_hessian_mat = sp.sparse.lil_matrix((i,c*c))
    for p in range(i):
            temp_mat = (np.transpose(s[p,:])*s[p,:]).todense().ravel(order='F')
            
            s_hessian_mat[p,:] = scipy.sparse.csr_matrix(temp_mat)
    print('yo')
    def F(x=None, z=None):
        if x is None: return 0, matrix(0.0, (c+bins,1))
        
        gm = np.transpose(np.array(x[c:]))
        y = s*x[:c]
        
        w = (gm+y*tm)
        
        term2 = np.array(N0)*np.array(np.exp(w))
        term1 = np.array(Nsm)*np.array(w)
        
        f = cvxopt.matrix(-np.sum(term1-term2))
        Df = np.zeros((1,c+bins))
        
        Nm = np.sum(Nsm,axis=0)
        
        Df_gm = Nm - sum(term2)
        
        Df_theta =  np.transpose((Nsm - term2)*tm)*s
    
        Df[0,:c] = Df_theta
        
        Df[0,c:] = Df_gm
        Df_cvx = cvxopt.matrix(-Df)
        if z is None: return f, Df_cvx
        H = np.zeros((c+bins,c+bins))
        
        #first do the theta_theta terms
                
        #first do N_0*tm**2, this will produce a ixm matrix where each term is Ni0 * tm**2.
        Inner_term = N0*tm_matrix_squared
        print Inner_term
        #now multiply by a column vector form of w, this will do the multiplication by y, and also sum over m.
        #you now have an ix1 matrix
        Inner_term2 = np.sum(np.array(Inner_term)*np.array(np.exp(w)),axis=1)
        print Inner_term2.shape
        print s_hessian_mat.shape
        #multiply by hessian matrix and sum over sequences...
     
        H_thetas_temp = Inner_term2*s_hessian_mat
        
        print H_thetas_temp.shape

        
        #now we need to reshape such to fill in these values
        H[:c,:c] = H_thetas_temp.reshape((c,c),order='F')
        
        
        #now do mixed terms with partials of gms and thetas
        #first multiply the N0 counts by the exponential term. This will give you a matrix of ixm
        Inner_term = np.array(N0)*np.array(np.matrix(np.exp(w)))
        #now use element wise multiplication to multiply by the tms, this will broadcasts their value down the rows
        Inner_term2 = np.array(tm)*Inner_term
        #now convert back to matrix form and multiply by s. This sums over sequence while multiplying by si, and
        #yeilds a matrix of dimension mxc
        H_gm_theta_temp = np.transpose(np.matrix(Inner_term2))*s
        H[c:c+bins,:c] = H_gm_theta_temp
        
        #now we are going to do the last bit, the partials with respect to the gms
        
        #we can use the same 'Inner term' as above N0*exp(gm+tm*sum(theta*s))
        #sum over s
        Temp_term = np.sum(Inner_term,axis=0)
        ident = np.identity(bins)
        #use array multiplication to broadcast multiply such that all off diagonal terms are 0 and
        #the on diagonal terms are equal to sum(Ns0*exp(gm+sum(theta*s)))
        H_gms = np.array(Temp_term)*ident
        H[c:c+bins,c:c+bins] = H_gms
        
        
        #now insure that H is symmetric across diagonal
        for q in range(c+bins):
            for k in range(q):
                H[k,q] = H[q,k]
        
        H = cvxopt.matrix(z[0]*H)            
                
        return f, Df_cvx, H
    return solvers.cp(F)['x']

In [51]:
import mpathic.numerics as numerics
df_pan = pd.io.parsers.read_csv('/media/bill/New_Volume/mpathic-master/data/formatted/sortseq/crp-wt-formatted.txt',delim_whitespace=True)
df_pan['seq'] = df_pan['seq'].str.slice(3,24)
seq_mat_array_pan,wtrow = numerics.dataset2mutarray(df_pan.copy(),'MAT')

cols_for_keep = [x for x in range(seq_mat_array_pan.shape[1]) if x not in np.nonzero(wtrow)[0]]
seq_mat_array_pan = seq_mat_array_pan.tocsc()
seq_mat2 = seq_mat_array_pan[:,cols_for_keep]
print seq_mat_array_pan.indptr

[     0      0   2582   5558  41966  41966  45151  48384  53202  53202
  56311  59845  64630  67495  70024  72834  72834  76283  79092  79092
  82947  85783  88186  91068  91068  94524  97564  97564 101308 101308
 104525 108150 112310 115949 119268 119268 123679 126736 129529 132675
 132675 135903 138648 141524 141524 141524 144677 148550 152975 156579
 159902 159902 163626 167878 167878 171855 176515 179484 182016 184802
 184802 188414 188414 191670 195733 195733 198765 202264 206534 210388
 210388 213803 218027 222011 224729 227882 227882 232132 232132 235731
 240175 240175 243362 246838 252170]


In [9]:
def test_iter(A, B):
    m,n1 = A.shape
    n2 = B.shape[1]
    Cshape = (m, n1*n2)
    #initialize our output sparse matrix data objects
    data = np.empty((m,),dtype=object)
    col =  np.empty((m,),dtype=object)
    row =  np.empty((m,),dtype=object)
    #do multiplication for each row
    for i,(a,b) in enumerate(zip(A, B)):
        #column indexes
        col1 = a.indices * n2
        col[i] = (col1[:,None]+b.indices).flatten()
        row[i] = np.full((a.nnz*b.nnz,), i)
        #all data will be 1's, as it is only true or false data
        data[i] = np.ones(len(col[i]))
    data = np.concatenate(data)
    col = np.concatenate(col)
    row = np.concatenate(row)
    return sparse.coo_matrix((data,(row,col)),shape=Cshape)

In [10]:
import cvxopt
import scipy as sp
print seq_mat2.shape
#q = seq_mat2.nonzero()
#emcvx = cvxopt.spmatrix(seq_mat2.data,q[0],q[1])
N0 = np.matrix(df_pan['ct_0']).T
Nsm = np.matrix(df_pan['ct_1']).T
tm = np.array([1])
output = robls(seq_mat2,wtrow,N0,Nsm,tm)

(45050, 9)


NameError: name 'robls' is not defined

In [97]:
def reverse_parameterization(output,wtrow,seq_dict,bins=1):
    
    output = output[:-bins]
    cols_for_keep = np.where(wtrow==False)[0]
    df_out = np.zeros(len(wtrow))
    df_out.flat[cols_for_keep] = output
    
    df_out2 = df_out.reshape((len(seq_dict),len(df_out)/len(seq_dict)),order='F')
    return df_out2

In [99]:
seq_dict,inv_dict = utils.choose_dict('dna')
out_df = reverse_parameterization(output,wtrow,seq_dict)
print out_df

[[ 0.          0.          0.          0.41604718  0.23452911 -0.04367564
   0.51505114  0.         -0.03732122  0.1913667  -0.06025818  0.
  -0.17294478 -0.35119965  0.2082949   0.25656217  0.          0.09347678
  -0.72590348 -0.50077798  0.         -0.0611443 ]
 [ 0.13677706  0.12417908 -0.02348784  0.27680913  0.50582905  0.02215633
   0.51209808  0.26520922 -0.01754959  0.09950115  0.01933881 -0.06446618
   0.15977853  0.          0.25777742  0.          0.20354249  0.
   0.03097377  0.          0.49971326 -0.06643488]
 [ 0.14240891  0.1996765   0.22396656  0.47146383  0.          0.08284538
   0.          0.34363172  0.         -0.08275695  0.32121498  0.07233987
   0.          0.02248156  0.18067027  0.45802364  0.02199375  0.42078577
  -0.18256252 -0.12118496  0.37408902  0.19966501]
 [-0.1398095  -0.23432475 -0.00866126  0.          0.23571744  0.
   0.44838763  0.26019653 -0.2820036   0.          0.          0.02877254
  -0.23679748 -0.36038511  0.          0.11045232  0.1868

In [11]:
import mpathic.utils as utils
seq_dict,inv_dict = utils.choose_dict('dna')
par_seq_dict = {v:k for v,k in seq_dict.items() if k != (len(seq_dict)-1)}
def genweightandmat(weights_df,seq_dict,dicttype,means=None,modeltype='MAT'):
    '''For use with learn_model, linear regressions. Generates a flattened
         matrix representation of the sequences, with corresponding batch and 
         number of counts (in sequence weighting vector)'''
    type_name_dict = {'dna':'seq','rna':'seq_rna','protein':'seq_pro'}
    seq_col_name = type_name_dict[dicttype]
    n_seqs = len(weights_df[seq_col_name])    
    mut_region_length = len(weights_df[seq_col_name][0])
    binheaders = get_column_headers(weights_df)
    nbins=len(binheaders)
    
    if modeltype == 'MAT':
         lasso_mat = sp.sparse.lil_matrix((n_seqs,len(seq_dict)*mut_region_length))
    elif modeltype == 'NBR':
         lasso_mat = sp.sparse.lil_matrix((n_seqs,len(seq_dict)*(mut_region_length-1)))
    counter = 0
    sample_weights = np.zeros(n_seqs)
    batch = np.zeros(n_seqs)
    
    for i,s in enumerate(weights_df[seq_col_name]):
             if modeltype == 'MAT':
                 lasso_mat[i,:] = utils.seq2matsparse(s,seq_dict)
             elif modeltype == 'NBR':
                 lasso_mat[i,:] = utils.seq2mat_sparse_neighbor(s,seq_dict)
             else:
                  raise ValueError('Incorrect Model Type')
             '''Find mean batch number for each sequence. If means is supplied,
                 instead find a weighted average.'''   
             if isinstance(means,pd.Series):
                 batch[i] = np.sum([weights_df[z][i]*means[z] for z in means.index]/(weights_df['ct'][i]))
             else:  
                 batch[i] = np.sum([weights_df[binheaders[z]][i]*z for z in range(nbins)]/weights_df['ct'][i])
             sample_weights[i] = weights_df['ct'][i]
             
    return sp.sparse.csr_matrix(lasso_mat),batch,sample_weights

In [ ]:
import numpy.linalg
import mpathic
df_lasso,batch,weights
a = np.transpose(seq_mat_array_pan)
a.shape

In [52]:
seq_mat_array_pan,wtrow = numerics.dataset2mutarray(df_pan.copy(),'NBR')

In [66]:
print(df_pan.seq[0])
print seq_mat_array_pan.sum(axis=0)

AAAAGTGAGATGGCAATCTAA
[[    0   202   213   311  1931   177   208   266  2201   213   240   322
  27324  2593  2572  3919     0  2301  2651  3593  2369   225   247   344
   2360   228   278   367  3624   355   358   481  2129  1895  2056     0
    222   153   210  2524   226   192   224  2892   288   289   320  3888
    243   208  2122   292   197   181  1930   221   211   169  2166   264
   2798  2251     0  3078   209   199   203  2838   229   174   187  2219
   2139  1828  2179     0   259   202   313  3081   244   204  2153   235
    211   179  1821   192   244   199  2197   242  2757  2458     0  3075
   2539   263   307   347  2268   226   239   307     0  2458  2762  3147
   2798   270   317   359  2713  2558     0  3366   311   202  2419   285
    269   239  2750   367   346   320  3101   393   245   220   321  2853
    220   219   234  2646  2259  2072  2329     0   333   282   262  3534
    214   186   183  2474   219   178   161  2235   217   176   203  2550
   2578  2205  2

In [23]:
seq_dict,inv_dict = utils.choose_dict('dna',modeltype='NBR')
par_seq_dict = {v:k for v,k in seq_dict.items() if k != (len(seq_dict)-1)}
raveledmat,batch,sw = utils.genweightandmat(
                                  df_pan,par_seq_dict,'dna',means=None,modeltype='NBR')

In [29]:
np.linalg.matrix_rank(raveledmat.todense())

27

In [15]:
seq_mat_array_pan.sum(axis=0)

matrix([[    0,   202,   213,   311,  1931,   177,   208,   266,  2201,
           213,   240,   322, 27324,  2593,  2572,  3919,     0,  2301,
          2651,  3593,  2369,   225,   247,   344,  2360,   228,   278,
           367,  3624,   355,   358,   481,  2129,  1895,  2056,     0,
           222,   153,   210,  2524,   226,   192,   224,  2892,   288,
           289,   320,  3888,   243,   208,  2122,   292,   197,   181,
          1930,   221,   211,   169,  2166,   264,  2798,  2251,     0,
          3078,   209,   199,   203,  2838,   229,   174,   187,  2219,
          2139,  1828,  2179,     0,   259,   202,   313,  3081,   244,
           204,  2153,   235,   211,   179,  1821,   192,   244,   199,
          2197,   242,  2757,  2458,     0,  3075,  2539,   263,   307,
           347,  2268,   226,   239,   307,     0,  2458,  2762,  3147,
          2798,   270,   317,   359,  2713,  2558,     0,  3366,   311,
           202,  2419,   285,   269,   239,  2750,   367,   346,

In [53]:
no_reps = np.sum(np.matrix(df_pan['ct_0'])*seq_mat_array_pan,axis=0)
print len(np.nonzero(no_reps)[1])
#cols_for_keep = [x for x in range(seq_mat.shape[1]) if x not in np.nonzero(wtrow)[0]]
cols_for_keep = [x for x in range(seq_mat_array_pan.shape[1]) if x in np.nonzero(no_reps)[1]]

300


In [54]:
seq_mat_array_pan2 = seq_mat_array_pan[:,cols_for_keep]

In [55]:
np.linalg.matrix_rank(seq_mat_array_pan2.todense())

243

In [58]:
seq_mat_array_pan2.shape

(45050, 300)

In [28]:
np.sum(np.matrix(df_pan['ct_0'])*seq_mat_array_pan,axis=0)

matrix([[    0.,    31.,    29.,    39.,   251.,    27.,    33.,    31.,
           287.,    19.,    28.,    35.,  3449.,   348.,   363.,   569.,
             0.,   304.,   334.,   423.,   338.,    17.,    26.,    44.,
           345.,    24.,    20.,    64.,   529.,    44.,    47.,    54.,
           251.,   246.,   283.,     0.,    36.,    18.,    33.,   302.,
            21.,    25.,    24.,   357.,    31.,    39.,    40.,   475.,
            25.,    25.,   267.,    22.,    27.,    26.,   244.,    31.,
            36.,    27.,   277.,    40.,   385.,   318.,     0.,   456.,
            31.,    31.,    20.,   391.,    36.,    16.,    33.,   311.,
           317.,   308.,   352.,     0.,    36.,    23.,    39.,   451.,
            44.,    27.,   323.,    26.,    25.,    27.,   298.,    28.,
            46.,    35.,   333.,    30.,   377.,   343.,     0.,   430.,
           351.,    44.,    49.,    48.,   321.,    24.,    39.,    48.,
             0.,   331.,   371.,   453.,   393.,   

In [24]:
seq_mat_array_pan2.sum(axis=0)

matrix([[  202,   213,   311,  1931,   177,   208,   266,  2201,   213,
           240,   322, 27324,  2593,  2572,  3919,  2301,  2651,  3593,
          2369,   225,   247,   344,  2360,   228,   278,   367,  3624,
           355,   358,   481,  2129,  1895,  2056,   222,   153,   210,
          2524,   226,   192,   224,  2892,   288,   289,   320,  3888,
           243,   208,  2122,   292,   197,   181,  1930,   221,   211,
           169,  2166,   264,  2798,  2251,  3078,   209,   199,   203,
          2838,   229,   174,   187,  2219,  2139,  1828,  2179,   259,
           202,   313,  3081,   244,   204,  2153,   235,   211,   179,
          1821,   192,   244,   199,  2197,   242,  2757,  2458,  3075,
          2539,   263,   307,   347,  2268,   226,   239,   307,  2458,
          2762,  3147,  2798,   270,   317,   359,  2713,  2558,  3366,
           311,   202,  2419,   285,   269,   239,  2750,   367,   346,
           320,  3101,   393,   245,   220,   321,  2853,   220,

In [27]:
seq_mat_array_pan2[0,:].todense()

matrix([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
         0, 0, 0, 0, 0, 0